In [3]:
import sys # 시스템 : 쓸수도 있고 안쓸수도 있지만 일단 가져오기
import os  # 시스템 : 쓸수도 있고 안쓸수도 있지만 일단 가져오기

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리 # 아직안씀
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
#from tqdm import tqdm_notebook   # tqdm이전 버전의 사용 방법. 버전이 완전히 바뀌면 아래 코드만을 사용해야 합니다.
from tqdm.notebook import tqdm    # for문 돌릴 때 진행상황을 %게이지로알려줍니다.

In [7]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"       # 윈도우는 "chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 melon(음원사이트)
driver.get('http://www.melon.com')
time.sleep(2)  # 2초간 정지

In [ ]:
# 멜론차트 클릭
driver.find_element_by_css_selector(".menu_bg.menu01").click()
time.sleep(2)

# 일간차트 클릭
driver.find_element_by_css_selector(".menu_chart.m2").click()


# 크롤링 코드

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 100  # 수집할 글 갯수 정하기

for i in range(0,number):
    # 곡정보 더보기 버튼 클릭
    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
    more_info_list[i].click()
    time.sleep(1)
    # 크롤링
    try : 
        music_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
        time.sleep(1)
        
        # 제목 크롤링
        overlays = ".song_name"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text

        # 가수 크롤링
        overlays = ".artist"                              
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        artist_name = nick.text
        time.sleep(1)

        # 발매날짜 크롤링
        overlays = ".list"                                
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')         # datetime
        Album_date = meta_info[3]
        
        # 좋아요 수
        overlays = "d_like_count"
        like_count = driver.find_element_by_id(overlays)
        LIKE = like_count.text
        
        # 앨범이름 크롤링
        overlays = ".list"
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')
        Album_name = meta_info[1]
        
        # 현재 곡 장르 크롤링
        overlays = ".list"
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')
        Album_Genre = meta_info[5]
        
        # 더보기 버튼 클릭
        driver.find_element_by_css_selector(".button_more.arrow_d").click()
        time.sleep(3)
        
        # 가사 크롤링
        overlays = ".lyric.on"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents
        
        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        music_info['title'] = title
        music_info['artist'] = artist_name
        music_info['Album_date'] = Album_date
        music_info['LIKE'] = LIKE
        music_info['Album_name'] = Album_name
        music_info['Album_Genre'] = Album_Genre
        music_info['Lyric'] = content_str
        time.sleep(1)
        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = music_info
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 뒤로가기
        driver.back()  
    
    # 에러처리 : 에러나면 뒤로가기하고 다음 글(i+1)로 이동합니다.
    except:
        driver.back()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기  - 에러날 경우 데이터 손실 위험 감소
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("Music_melon.xlsx", encoding='utf-8-sig')
        time.sleep(3)

# 저장하기
result_df.to_excel("Music_melon.xlsx", encoding='utf-8-sig')
time.sleep(3)
driver.quit()

print('수집한 글 갯수: ', len(dict))

0 TOMBOY
1 GANADARA (Feat. 아이유)
2 Feel My Rhythm
3 사랑인가 봐
4 취중고백
5 INVU
6 듣고 싶을까
7 사랑은 늘 도망가
8 RUN2U
9 언제나 사랑해
10 다정히 내 이름을 부르면
11 SMILEY (Feat. BIBI)
12 ELEVEN
13 회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)
14 신호등
15 존재만으로
16 어제처럼
17 Step Back
18 STAY
19 너, 너 (N번째 연애 X 휘인 (Whee In))
20 내 생에 아름다운
21 드라마
22 리무진 (Feat. MINO) (Prod. GRAY)
23 strawberry moon
24 버퍼링 (Glitch Mode)
25 Next Level
26 사랑인걸
27 눈이 오잖아(Feat.헤이즈)
28 호랑수월가
30 Counting Stars (Feat. Beenzino)
31 Weekend
32 너를 생각해
33 Dreams Come True
34 Starlight
35 라일락
36 바라만 본다
37 Savage
38 Dynamite
39 OHAYO MY NIGHT
40 Butter
41 MY BAG
42 만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. TOIL)
43 Celebrity
44 찰나가 영원이 될 때 (The Eternal Moment)
45 이제 나만 믿어요
46 겨울잠
47 사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)
48 Permission to Dance
49 With
50 아주, 천천히
51 서랍
52 Can't Control Myself
53 Bad Habits
54 헤픈 우연
55 내 손을 잡아
56 꽃 (With 김민석 of 멜로망스)
57 Blueming
58 밤하늘의 별을(2020)
59 모든 날, 모든 순간 (Every day, Every Moment)
60 문득
61 벚꽃이 피면 우리 그만 헤어져
62 봄날
63 너의 모든 순

In [ ]:
result_df

In [ ]:
for i in range(100) :
    try :
        result_df['Lyric'][i] = result_df['Lyric'][i].replace('\n',' ')
    except :
        continue

In [ ]:
result_df['Lyric']

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from afinn import Afinn
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

dataset = pd.read_csv("sentiment_analysis.csv",encoding = 'utf-8')
dataset.head()

In [ ]:
sentimential_list = {'행복' : 0, '기쁨': 0, '분노' : 1, '불안' : 2, '당황' : 2, '슬픔' : 3, '상처' : 3}
dataset['label'] = dataset['sentiment_big'].map(sentimential_list)
dataset.head()

In [ ]:
import re
dataset['text'] = dataset['person1'].map(str) + dataset['system1'].map(str) + dataset['person2'].map(str)+ dataset['system2'].map(str) + dataset['person3'].map(str) + dataset['system3'].map(str)
new_data = {"text" : "ㅎㅎ","label": 0}
dataset = dataset.append(new_data, ignore_index = True)

x_data = dataset['text']
y_data = dataset['label'].values

x_data = x_data.apply(lambda x : re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', " ", x)).values
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, stratify = y_data, test_size = 0.2, random_state = 42)


from konlpy.tag import Okt

okt = Okt()
def tw_tokenizer(text):
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [ ]:
# label 감정별 데이터 분포

dataset['label'].value_counts()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
import time

In [ ]:
from lightgbm import LGBMClassifier

def light_gbm_train(x_train_okt, x_test_okt, y_train, y_test):
    print("LGBM Classifier\n")
    start_time = time.time()
    lgbm_clf = LGBMClassifier(n_estimators = 400, n_jobs=-1, reg_alpha = 0.9, reg_lambda = 0.2)
    lgbm_clf.fit(x_train_okt, y_train)
    preds = lgbm_clf.predict(x_test_okt)
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    
    precision = precision_score(y_test, preds, average='weighted')
    recall = recall_score(y_test, preds, average='weighted')
    f1 = f1_score(y_test, preds, average='weighted')
    
    print(confusion)
    print('정확도: {0:.4f}\n정밀도: {1:.4f}\n재현율: {2:.4f}\nf1: {3:.4f}'.format(accuracy,precision,recall,f1))
    print(f'fit time : {time.time() - start_time}')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range = (1, 2), min_df = 3, max_df = 0.9, tokenizer = okt.morphs, token_pattern = None)
tfidf.fit(x_train)
x_train_okt = tfidf.transform(x_train)
x_test_okt = tfidf.transform(x_test)

light_gbm_train(x_train_okt, x_test_okt, y_train, y_test)

In [ ]:
lgbm_clf = LGBMClassifier(n_estimators = 400, n_jobs=-1, reg_alpha = 0.9, reg_lambda = 0.2)
lgbm_clf.fit(x_train_okt, y_train)
light_gbm_train(x_train_okt, x_test_okt, y_train, y_test)

In [ ]:
lyric_sentiment_list = []

In [ ]:
for i in range(0,result_df.shape[0]):
    try :
        if result_df['Lyric'][i] != [] :
            lyric_sentiment = list(lgbm_clf.predict(tfidf.transform([result_df['Lyric'][i]])))
            lyric_sentiment_list.append(lyric_sentiment[0])
    except:
        continue

In [ ]:
lyric_sentiment_list

In [ ]:
lyric_sentiment_df = pd.DataFrame(lyric_sentiment_list)
lyric_sentiment_df.columns = ["label"]
lyric_sentiment_df

In [ ]:
result_df

## 0 : 기쁨 행복
## 1 : 분노 
## 2: 불안 당황
## 3: 슬픔 상처

In [ ]:
Liryc_df = pd.merge(result_df,lyric_sentiment_df,left_index= True, right_index=True )
Liryc_df

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup
import time 
import re
import json 
import pandas as pd
from datetime import datetime

In [ ]:
## id 와 password 입력

user_id=""
user_passwd=""

In [ ]:
login_option="instagram" # facebook or instagram 
driver_path="chromedriver.exe" 
instagram_id_name="username" 
instagram_pw_name="password"
instagram_login_btn=".sqdOP.L3NKy.y3zKF "
instagram_information_save_name = ".sqdOP.yWX7d.y3zKF"
instagram_alarm_name = ".aOOlW.HoLwm"
is_information_save = False
is_alarm_setting = False

driver = webdriver.Chrome("./chromedriver")

In [ ]:
login_url = "https://www.instagram.com/accounts/login/"
driver.get(login_url)

if login_option == "instagram": 
    time.sleep(2)
    try: 
        instagram_id_form = driver.find_element_by_name(instagram_id_name) 
        instagram_id_form.send_keys(user_id) 
        
        instagram_pw_form = driver.find_element_by_name(instagram_pw_name) 
        instagram_pw_form.send_keys(user_passwd) 
        
        login_ok_button = driver.find_element_by_css_selector(instagram_login_btn) 
        login_ok_button.click() 
        is_login_success = True
        
        time.sleep(3)
        if is_information_save == False and is_alarm_setting == False:
            information_save_btn = driver.find_element_by_css_selector(instagram_information_save_name)
            information_save_btn.click()
            is_information_save = True
            
            alarm_setting_btn = driver.find_element_by_css_selector(instagram_alarm_name)
            alarm_setting_btn.click()
            is_alarm_setting = True
            
    except: 
        print("instagram login fail") 
        is_login_success = False

In [ ]:
## 위 내용까지 로그인

my_account_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/span"
my_account_button = driver.find_element_by_xpath(my_account_xpath)
my_account_button.click()

my_profile_full_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/div[2]/div[2]/div[2]/a[1]/div"
my_profile_button = driver.find_element_by_xpath(my_profile_full_xpath)
my_profile_button.click()

time.sleep(1.5)
# 게시물 개수
post_count = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/header/section/ul/li[1]/div/span").text
post_count = int(post_count)

print("게시물 수 : ",post_count)

insta_df = pd.DataFrame()
new_data = {"date" : "날짜", "place":"장소","content":"본문내용","hashtag":"해시태그","comment":"댓글내용"}
insta_df = insta_df.append(new_data,ignore_index=True)
insta_df.head()

In [ ]:
# 첫 게시물
first_post_css = "a > div.eLAPa > div._9AhH0"
first_post_button = driver.find_element_by_css_selector(first_post_css)
first_post_button.click()

for i in range(post_count) :
    # 날짜 데이터 크롤링
    time.sleep(2)
    date = driver.find_element_by_css_selector('time._1o9PC')
    date_info = pd.to_datetime(date.get_attribute('datetime')).normalize()
    date = str(date_info.year) + "년 " + str(date_info.month) + "월 " + str(date_info.day) +"일"

    # 장소 크롤링
    time.sleep(0.5)
    try : 
        place = driver.find_element_by_css_selector("body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.UE9AK > div > header > div.o-MQd.z8cbW > div.M30cS > div > div.JF9hh > div > a").text
    except : 
        place = None

    try : 
        post_content = driver.find_element_by_xpath("/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/div/li/div/div/div[2]/div[1]/span").text    
    except :
        post_content = None
    post_content_list =[]
    post_content_list.append(post_content)
   
    # 본문 속 해시태그 정제
    if post_content != None :
        post_content_tag= []
        post_content = re.findall("#[A-Za-z0-9가-힣]+", post_content)
        content = ''.join(post_content).replace("#"," ") # "# 제거"
        content_split = content.split()
        
        for tag in content_split :
            post_content_tag.append(tag)
    
    # 댓글 크롤링
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    comment_count = 0

    # 댓글 있는지 확인
    try  :
        ul_list = soup.find_all('ul',  'Mr508')
        comment_count = len(ul_list)
    except :
        # 댓글 없으면
        comment_count = 0
    else : 
        # 댓글 개수 적용하여 대댓글 열기버튼 xpath 추출
        for i in range(1,comment_count+1) :
            comment_list = []
            button_xpath = "/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[" + str(i) + "]/li/ul/li/div/button"
            try : 
                # 대댓글 열기버튼 있으면 클릭
                driver.find_element_by_xpath(button_xpath).click()
                time.sleep(1)
                button_check = driver.find_element_by_class_name("EizgU").text
                while "답글 숨기기" not in button_check : 
                    driver.find_element_by_xpath(button_xpath).click()
                    button_check = driver.find_element_by_class_name("EizgU").text
                    
            except :
                button_xpath = None
            
        # 대댓글이 모두 열려있는 상태에서 html parser        
        # 추가 댓글이 모두 열려있는 상태에서 html parser        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ul_list = soup.find_all("ul","Mr508")
        for d in ul_list :
            user_id = d.find("h3","_6lAjh").find("a").get_text()

            comment = d.find("div","C4VMK").find("div","MOdxS").get_text()

            comment_list.append(user_id+":"+comment)
            try :
                re_ul_list = d.find_all("ul","TCSYW")
                for dd in re_ul_list :
                    #print("[ 답글시작 ]")
                    try : 
                        re_div_list = dd.find_all("div","ZyFrc")
                        for ddd in re_div_list :
                            #print("[답글의 답글 시작]")
                            comment_user_id = ddd.find("h3","_6lAjh").find("a").get_text()
                            
                            comment_comment = ddd.find("div","C4VMK").find("div","MOdxS").get_text()

                            comment_list.append(comment_user_id+":"+comment_comment)
                    except : 
                        re_div_list =""
            except : 
                re_ul_list = ""

    new_data = {"date" : date, "place":place, "content" : post_content_list,"hashtag" : post_content_tag,"comment" : comment_list}
    insta_df = insta_df.append(new_data,ignore_index=True)
    # 다음 버튼
    try :
        next_post_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.Z2Inc._7c9RR > div > div.l8mY4.feth3 > button")
        next_post_button.click()
        time.sleep(0.5)
    except :
        # 닫기 버튼    
        exit_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.NOTWr > button > div > svg")
        exit_button.click()
        break

In [ ]:
insta_df

In [ ]:
comment_re_list = ['댓글내용 정제']   

In [ ]:
for k in range(1,insta_df['comment'].shape[0]) :
    str = " "
    strstr=" "
    for i in range(len(insta_df['comment'][k])) :
        str = insta_df['comment'][k][i].split(":")
        str = str[1]
        str_split = str.split("@")
        if(len(str_split) != 1) :
            index = str_split[1].find(" ")
            str_split = str_split[1]
            str_split = str_split[index:]
            str = str_split
        if(str.find(" ") == 0) :
            str = str[1:]
        str = str.replace("#"," ")
        strstr += str
    comment_re_list.append(strstr)

In [ ]:
comment_re_df = pd.DataFrame(comment_re_list)
comment_re_df.columns = ["comment_re"]
comment_re_df

In [ ]:
insta_df = pd.merge(insta_df,comment_re_df,left_index= True, right_index=True )
insta_df

In [ ]:
comment_sentiment_list = ['감정분석결과']

In [ ]:
for i in range(1,insta_df.shape[0]):
    if insta_df['comment_re'][i] != [] :
        comment_sentiment=list(lgbm_clf.predict(tfidf.transform(insta_df['comment'][i])))
    if insta_df['content'][i] != [] :
        comment_sentiment=list(lgbm_clf.predict(tfidf.transform(insta_df['content'][i])))
    if insta_df['hashtag'][i] != [] :
        comment_sentiment=list(lgbm_clf.predict(tfidf.transform(insta_df['hashtag'][i])))
    comment_sentiment_list.append(max(comment_sentiment,key = comment_sentiment.count))
comment_sentiment_list

In [ ]:
comment_sentiment_df = pd.DataFrame(comment_sentiment_list)
comment_sentiment_df.columns = ["label"]
comment_sentiment_df

In [ ]:
insta_df = pd.merge(insta_df,comment_sentiment_df,left_index= True, right_index=True )
insta_df